In [49]:
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import dash
import json

In [2]:
df_vdem = pd.read_csv("V-Dem-CY-Full+Others-v13.csv")

/var/folders/jr/yk_drf0n3ys0hbglvhv_fsth0000gn/T/ipykernel_4119/3707769492.py:1: DtypeWarning: Columns (364,365,366,394,410,821,831,832,930,1235,1252,1481,3063,3282,3283,3351,3353,3357,3358,3360,3361,3362) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vdem = pd.read_csv("V-Dem-CY-Full+Others-v13.csv")


In [3]:
df_regime_1 = df_vdem.pivot(index='country_name', columns='year', values='v2x_regime')
df_dem_1 = df_vdem.pivot(index='country_name', columns='year', values='e_boix_regime')
df_polity_1 = df_vdem.pivot(index='country_name', columns='year', values='e_polity2')

In [4]:
df_regime_2 = df_regime_1.drop(columns=list(range(1789, 1991))+ [2019, 2020, 2021, 2022])
df_dem_2 = df_dem_1.drop(columns=list(range(1789, 1991)) + [2019, 2020, 2021, 2022])
df_polity_2 = df_polity_1.drop(columns=list(range(1789, 1991)) + [2019, 2020, 2021, 2022])

df_regime_3 = df_regime_2.dropna()
df_dem_3 = df_dem_2.dropna()
df_polity_3 = df_polity_2.dropna()

In [5]:
regime_transitions_list = []
for index, row in df_regime_3.iterrows():
    country_name = index
    transition_started = False
    
    for year in df_regime_3.columns[:-1]:
        current_value = row[year]
        
        if (current_value >= 2) and all(row[year + i] < 2 for i in range(1, len(df_regime_3.columns) - df_regime_3.columns.get_loc(year))):
            transition_started = True
            transition_start_year = year

        elif transition_started and all(row[year + i] < 2 for i in range(1, len(df_regime_3.columns) - df_regime_3.columns.get_loc(year))):
            transition_info = {"country": country_name, "start_year": transition_start_year}
            regime_transitions_list.append(transition_info)
            transition_started = False
            break

dem_transitions_list = []
for index, row in df_dem_3.iterrows():
    country_name = index
    transition_started = False
    
    for year in df_dem_3.columns[:-1]:
        current_value = row[year]
        
        if current_value == 1.0 and all(row[year + i] == 0.0 for i in range(1, len(df_dem_3.columns) - df_dem_3.columns.get_loc(year) - 1)):
            transition_started = True
            transition_start_year = year

        elif transition_started and all(row[year + i] == 0.0 for i in range(1, len(df_dem_3.columns) - df_dem_3.columns.get_loc(year) - 1)):
            transition_info = {"country": country_name, "start_year": transition_start_year}
            dem_transitions_list.append(transition_info)
            transition_started = False
            break
            
polity_transitions_list = []
for index, row in df_polity_3.iterrows():
    country_name = index
    transition_started = False
    
    for year in df_polity_3.columns[:-1]:
        current_value = row[year]
        
        if (current_value >= 7) and all(row[year + i] < 7 for i in range(1, len(df_polity_3.columns) - df_polity_3.columns.get_loc(year) - 1)):
            transition_started = True
            transition_start_year = year

        elif transition_started and all(row[year + i] < 7 for i in range(1, len(df_polity_3.columns) - df_polity_3.columns.get_loc(year) - 1)):
            transition_info = {"country": country_name, "start_year": transition_start_year}
            dem_transitions_list.append(transition_info)
            transition_started = False
            break

regime_set = {entry["country"] for entry in regime_transitions_list}
dem_set = {entry["country"] for entry in dem_transitions_list}
polity_set = {entry["country"] for entry in polity_transitions_list}

intersection_set_1 = regime_set.intersection(dem_set)
intersection_set_2 = regime_set.intersection(polity_set)
intersection_set_3 = dem_set.intersection(polity_set)

union_of_sets = (intersection_set_1.union(intersection_set_2)).union(intersection_set_3)

union_list = [{"country": entry} for entry in union_of_sets]

union_list_2 = [{"country": entry["country"], "start_year": entry["start_year"]} for entry in regime_transitions_list if entry["country"] in union_of_sets]

df_union_list_2 = pd.DataFrame(union_list_2)

In [6]:
df_wdi_1 = pd.read_excel("P_Data_Extract_From_World_Development_Indicators.xlsx")

In [7]:
country_names = [country_dict['country'] for country_dict in union_list]

df_wdi_2 = df_wdi_1[df_wdi_1['Country Name'].isin(country_names)]

In [8]:
df_wdi_2.replace({"..": np.nan, "...": np.nan}, inplace=True)
df_wdi_2.dropna(inplace=True)

/var/folders/jr/yk_drf0n3ys0hbglvhv_fsth0000gn/T/ipykernel_4119/3393385148.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wdi_2.replace({"..": np.nan, "...": np.nan}, inplace=True)
/var/folders/jr/yk_drf0n3ys0hbglvhv_fsth0000gn/T/ipykernel_4119/3393385148.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wdi_2.dropna(inplace=True)


In [9]:
common_indicators = set(df_wdi_2['Series Name'])
for country in df_wdi_2['Country Name'].unique():
    country_indicators = set(df_wdi_2[df_wdi_2['Country Name'] == country]['Series Name'])
    common_indicators = common_indicators.intersection(country_indicators)

df_wdi_3 = df_wdi_2[df_wdi_2['Series Name'].isin(common_indicators)]

In [10]:
df_wdi_melted = df_wdi_2.melt(id_vars=['Country Name', 'Series Name', 'Series Code'], var_name='Year', value_name='Value')

In [12]:
df_plots = pd.DataFrame([
    {"plot": "Line Plot"},
    {"plot": "Bivariable Scatter Plot"},
    {"plot": "Bivariable Scatter Plot (Log)"}
])

In [19]:
from dash import Dash, html, dcc, dash_table, callback, Output, Input

In [50]:
import pandas as pd
import numpy as np
import plotly.express as px
import dash
from dash import html, dcc, dash_table
from dash.dependencies import Input, Output

def line_plot(data, x_column, y_column, color_column, title):
    fig = px.line(data, x=x_column, y=y_column, color=color_column, title=title)
    fig.update_traces(mode="lines")    
    return fig

def scatter_plot(data, x_column, y_column, color_column, hover_column, animation_column, title, x_label, y_label):
    min_value_x = data[x_column].min()
    max_value_x = data[x_column].max()
    min_value_y = data[y_column].min()
    max_value_y = data[y_column].max()
    fig = px.scatter(data, x=x_column, y=y_column, color=color_column,
                     hover_name=hover_column, animation_frame=animation_column,
                     title=title, animation_group=hover_column,
                     labels={x_column: x_label, y_column: y_label})
    fig.update_traces(marker=dict(size=10))
    fig.update_layout(xaxis=dict(range=[min_value_x, max_value_x]), yaxis=dict(range=[min_value_y, max_value_y]))
    return fig

def log_scatter_plot(data, x_column, y_column, color_column, hover_column, animation_column, title, x_label, y_label):
    min_value_x = data[x_column].min()
    max_value_x = data[x_column].max()
    min_value_y = data[y_column].min()
    max_value_y = data[y_column].max()
    fig = px.scatter(data, x=x_column, y=y_column, color=color_column,
                     hover_name=hover_column, animation_frame=animation_column, log_x=True, log_y=True,
                     title=title, animation_group=hover_column,
                     labels={x_column: x_label, y_column: y_label})
    fig.update_traces(marker=dict(size=10))
    fig.update_layout(xaxis=dict(range=np.log10([min_value_x, max_value_x])),
                      yaxis=dict(range=np.log10([min_value_y, max_value_y])))
    return fig

app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div(children='Democracy Backsliding and Analysis', style={'textAlign': 'center', 'fontSize': 24, 'fontWeight': 'bold'}),
    dcc.Markdown('''
        #### Understanding Democracy Backsliding
        In examining democracy backsliding across countries spanning the years 1991 to 2018, 
        our project relies on democracy metrics sourced from the Varieties of Democracy (V-Dem) dataset, version 13. 
        A nation is categorized as a democratic backslider under two specific conditions: 
        firstly, it must have attained a notable level of democracy, and subsequently, 
        it should undergo significant erosion of democratic institutions. 
        Our approach to defining a significant level of democracy considers a country to have achieved this status 
        when meeting criteria such as a Regimes of the World (ROW) measure of equal to or greater than 2, 
        a Democracy (BMR) score of 1, and a Polity revised combined score equal to or greater than 7. 
        Similarly, our criteria for democratic decline inclusively encompass countries experiencing a 
        qualitative rating decline or identified as backsliders by at least two major democracy indices. 
        This involves a ROW measure of less than 2, a Democracy (BMR) score of 0, and a Polity revised combined score of less than 7. 
        Based on our approach produces eleven cases of democratic backsliding since the end of the Cold War.
    ''', style={'textAlign': 'justify'}),
    html.Div([
        dash_table.DataTable(
            data=df_union_list_2.to_dict('records'),
            columns=[
            {'name': 'Country', 'id': 'country'},
            {'name': 'Year of Democracy Backsliding', 'id': 'start_year'},
            ],
            style_table={'height': '800px', 'overflowY': 'auto', 'width': '400px', 'margin': '0px'},
            style_cell={'textAlign': 'center'}
        )
    ], style={'display': 'flex', 'justify-content': 'center', 'margin-bottom': '10px'}),

    dcc.Graph(
        figure=px.choropleth(
            df_union_list_2,
            locations='country',
            locationmode='country names',
            hover_name='country',
            color='start_year',
            color_continuous_scale=px.colors.sequential.Plasma,
            title='Democracy Consolidation and Backsliding (Year 1991 to 2019)',
            labels={'start_year': 'Year of Democracy Backsliding', 'country': 'Country'},
        ),
        style={'width': '100%', 'height': '800px', 'margin-top': '-350px'}),
    
    dcc.Markdown('''
        #### Economics Analysis
        In order to further understand economic background of the countries that have experienced democracy backsliding,
        you can select indictors to graph.
    ''', style={'textAlign': 'justify'}),
    
    dcc.Dropdown(
        id='graph-dropdown',
        options=[
            {'label': 'Line Plot', 'value': 'Line Plot'},
            {'label': 'Scatter Plot', 'value': 'Scatter Plot'},
            {'label': 'Scatter Plot (Log)', 'value': 'Scatter Plot (Log)'}
        ],
        value='Line Plot',
        style={'width': '50%', 'margin-top': '10px'}),

    dcc.Dropdown(
        id='indicator-dropdown',
        options=[{'label': indicator, 'value': indicator} for indicator in df_wdi_melted['Series Name'].unique()],
        value=df_wdi_melted['Series Name'].iloc[0],
        style={'width': '50%', 'margin-top': '10px'}
    ),

    dcc.Graph(id='selected-graph', style={'width': '100%', 'height': '600px'})
])

@app.callback(
    Output('selected-graph', 'figure'),
    [Input('graph-dropdown', 'value'),
     Input('indicator-dropdown', 'value')]
)
def update_graph(selected_graph, selected_indicator):
    if selected_graph == 'Line Plot':
        return line_plot(df_wdi_melted, 'Year', 'Value', 'Country Name', f'{selected_indicator} Over Time')

    elif selected_graph == 'Scatter Plot':
        return scatter_plot(df_wdi_melted, 'Year', 'Value', 'Country Name', 'Country Name', 'Year',
                            f'{selected_indicator} vs. Time', 'Year', selected_indicator)

    elif selected_graph == 'Scatter Plot (Log)':
        return log_scatter_plot(df_wdi_melted, 'Year', 'Value', 'Country Name', 'Country Name', 'Year',
                                f'{selected_indicator} vs. Time (Log Scale)', 'Year', selected_indicator)

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, open_browser=False)


/var/folders/jr/yk_drf0n3ys0hbglvhv_fsth0000gn/T/ipykernel_4119/856095971.py:37: RuntimeWarning:

invalid value encountered in log10

/var/folders/jr/yk_drf0n3ys0hbglvhv_fsth0000gn/T/ipykernel_4119/856095971.py:37: RuntimeWarning:

invalid value encountered in log10

/var/folders/jr/yk_drf0n3ys0hbglvhv_fsth0000gn/T/ipykernel_4119/856095971.py:37: RuntimeWarning:

invalid value encountered in log10

